In [1]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
from keras.layers import *
import pickle, threading
from keras.utils import to_categorical
import linecache
from keras.callbacks import Callback
import re
from keras.callbacks import ModelCheckpoint
from collections import Counter, defaultdict


epochs = 50
trainable=True

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
nb_sequence_length=50
nb_embedding_dims=300

In [3]:

sys.version_info[0]

3

In [4]:
class WeightsSaver(Callback):
    def __init__(self, model, N):
        self.model = model
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'german_lm_twitter_20m.h5'
            print("model saved %s" %self.batch)
            self.model.save_weights(name)
        self.batch += 1

In [5]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [ ]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [6]:
def twitter_tokenizer(textline:'takes line as an input'):
    
    textline = re.sub('@[\\w_]+', 'USER_MENTION', textline) # replaces @username with 'usermention'
    textline = re.sub('\\|LBR\\|', '', textline) # to replace linebreak
    textline = re.sub('\\.\\.\\.+', '...', textline) #????????
    textline = re.sub('!!+', '!!', textline)#?????????
    textline = re.sub('\\?\\?+', '??', textline)
    words = re.compile('[\\U00010000-\\U0010ffff]|[\\w-]+|[^ \\w\\U00010000-\\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    return(words)

In [ ]:
# creating the first 1m sentences for use
def create_train_data(filename:'name of original file', n_lines:'no of lines to use for training from the twitter data',
                     new_file_name:'name of file to create'):
    new_file = open(new_file_name,'wb')
    n_lines_read=0
    with open(filename,'rb') as f:
        for line in f:
            if n_lines_read == n_lines:
                break
            line = line.decode()
            ids, text = line.split('\t')
            n_lines_read+=1
            text = twitter_tokenizer(text)
            to_write = ' '.join(text)
            to_write+='\n'
#             print(to_write)
            new_file.write(to_write.encode())
            
#             assert n_lines_read<=10
    f.close()
    
            

In [ ]:
create_train_data('/home/jindal/notebooks/twitter_data/twitter-2017_de', 20000000, 
                  '/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter_20m')

In [7]:
window_size =51

In [8]:
# converts the sentences to a fixed size of 51
def generate_sequences(out_file, in_file ) -> None:
    output = open(out_file,'wb')
    with open(in_file,'rb') as f:
        for line in f:
            text = line.decode().split()
            for i in range(len(text)):
                if i+1 >= window_size:
                    temp = text[i-window_size+1:i+1]
                else:
                    temp = ['0' for i in range(window_size - (i +1))] + text[:i+1]
                string = ' '.join(temp)+' \n'
                output.write(string.encode())
    output.close()

In [ ]:
generate_sequences('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m',
                   '/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter_20m')

In [ ]:
# assert
with open('twitter_dataset_sequences_20m') as f:
    for line in f:
        assert len(line.split()) == 51

In [ ]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [ ]:
print(caseEmbeddings.shape)

In [ ]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [ ]:
print(characters)

In [ ]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [ ]:
# returns the vocab of a file along with a counter of the vocab
def get_vocab(infile:"inputfile"):
    toy_vocab ={}
    toy_vocab_counter = defaultdict(int)


    with open(infile) as f:
        for line in f:
    #         print(line)
            line = line.split()
    #         print(len(line))
            for word in line:
    #             word = word
    #             print(word)
                toy_vocab[word]=True
                toy_vocab_counter[word]+=1
    return toy_vocab, toy_vocab_counter

In [9]:
line = linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m', 19)

print(line)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 Altere Rauchen Fetisch Sex Session https :// 



In [ ]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [10]:
line_number=0
# lock = threading.Lock()

In [11]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [12]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    max_words = 50
    idx = 0
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    output_word = words[-1]
    if output_word in word_vectors_ft:
        output_wv = word_vectors_ft[output_word]
    else:
        output_wv = ft.get_word_vector(output_word.lower())
    return features_ft, output_wv

In [13]:
def sequential_generator(filename, batch_size):
    
    file_length = file_len(filename)
    shuffled_indexes = range(1, file_length + 1) # start from 1 because linecache is 1 indexed
    nb_sequence_length = 50
    nb_embedding_dims = 300
    
    index_position = 0
    
    batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims)) # levy goldberg: not needed
    batch_labels = np.zeros((batch_size, 2)) # 2 because of simple task
    while True:
        # print(len(features))
        output_labels = []
        
        for i in range(batch_size):
            line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            batch_features_ft[i], output_wv = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
#             batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            output_labels.append(output_wv)
            index_position += 1
    
            if index_position == file_length:
                # shuffle indexes again
                shuffled_indexes = range(1, file_length + 1)
#                 shuffle(shuffled_indexes)
                index_position = 0
                break
        yield ([batch_features_ft], np.array(output_labels))


In [ ]:
def my_generator(file :"input training file", batch_size, total_lines):
    
    ft_dictionary={}
    global line_number, lock
        
    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []

        for i in range(batch_size):

            index = line_number%total_lines +1
            line_number+=1
            line = linecache.getline(file, index)
            line = line.split()
            temp_casing = []
            temp_char=[]
            temp_word=[]
            if len(line)!=51:
                print(index)
                continue
            for word in line[:-1]:
                casing =getCasing(word, case2Idx)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
                temp_char.append(temp_char2)
                if word.lower() in ft_dictionary:
                    word_vector = ft_dictionary[word.lower()]
                else:
                    wv = ft.get_word_vector(word.lower())
                    ft_dictionary[word.lower()] = wv
                word_vector = ft_dictionary[word.lower()]
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(ft.get_word_vector(line[-1]))
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

In [ ]:
for inp, output in my_generator('toy_dataset_sequences_shuffled',10):
    continue

In [14]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [16]:
model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(25, activation = "softmax")(model_concatenated)
model = Model(model_input_embedding, model_output)
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 50, 200)      320800      input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_6 (LeakyReLU)       (None, 50, 200)      0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 48, 200)      120200      leaky_re_lu_6[0][0]              
__________________________________________________________________________________________________
conv1d_5 (

In [ ]:
filter_sizes = (3, 4, 5)
model_input_ft = Input(shape = (nb_sequence_length, nb_embedding_dims))
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(model_input_ft)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.8)(conv)
    conv_blocks.append(conv)
lstm_block = LSTM(100)(model_input_ft) # , dropout = 0.5, recurrent_dropout = 0.5
lstm_block = LeakyReLU()(lstm_block)
model_concatenated = concatenate([lstm_block, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
model_concatenated = Dense(64)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(300, activation = "softmax")(model_concatenated)
model = Model([model_input_ft], model_output)
model.compile(loss='cosine_proximity', optimizer='nadam', metrics = ['accuracy'])
model.summary()

In [17]:
samples_per_epoch = file_len("/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m")
epochs = 1
batch_size = 64
steps_per_epoch = samples_per_epoch // batch_size
checkpoint = ModelCheckpoint('lm_twitter_20m.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [18]:

try:
    model.fit_generator(
        sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_20m', batch_size), 
        steps_per_epoch=steps_per_epoch, epochs=epochs,callbacks=[WeightsSaver(model, 200)]
        )
except Exception as e:
    model.save('german_lm_twitter_20m.h5')

Epoch 1/1


In [ ]:
words_input = Input(shape=(None, 300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(300, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
model.compile(loss='cosine_proximity', optimizer='nadam', metrics=['accuracy'])
model.summary()
# plot_model(model, to_file='model.png')

In [ ]:
print(label2)

In [ ]:
line_number=0
file_name = '/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_one_million'
try:
    model.fit_generator(my_generator(file_name, 64, file_len(file_name)), epochs=1, 
                        steps_per_epoch=file_len(file_name)//64, callbacks=[WeightsSaver(model, 20000)])
except Exception as e:
    model.save('german_lm_twitter.h5')
model.save('german_lm_twitter.h5')